In [1]:
# from langchain.chat_models import ChatOpenAI # -> ChatOpenAI는 langchain_openai에서 import하기
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

# from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings # -> OpenAIEmbeddings는 langchain_openai에서 import하기
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore

from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# vectorstore = Chroma.from_documents(docs, cached_embeddings)
vectorstore = FAISS.from_documents(docs, cached_embeddings)

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You ara a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

# retirever는 한 개의 string을 입력받고 document들의 list를 출력함
retriever = vectorstore.as_retriever()

# 기본적인 LLMChain 사용
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
# RunnablePassthrough()는 chain을 invoke할 때 입력된 쿼리를 그대로 입력할 수 있게 함
# 입력된 쿼리를 RunnablePassthrough()가 위치한 곳에 입력받을 수 있게 됨

In [3]:
chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is a dilapidated apartment complex located in Airstrip One, the third most populous province of Oceania. The building is described as having a gritty and unpleasant atmosphere, with a hallway that smells of boiled cabbage and old rag mats. The flat is seven flights up and is equipped with a telescreen, which is a device used for surveillance and communication by the Party.', response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 1959, 'total_tokens': 2049}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_0ccba42292', 'finish_reason': 'stop', 'logprobs': None}, id='run-0eb5f7d4-014b-4346-9bb7-7a1922f76114-0', usage_metadata={'input_tokens': 1959, 'output_tokens': 90, 'total_tokens': 2049})